Set up

In [ ]:
import locale
locale.getpreferredencoding = lambda : "UTF-8"
!pip install -q openai-whisper ffmpeg

Code, import libraries, choose whisper model, record


In [ ]:
from IPython.display import Javascript
from google.colab import output
from base64 import b64decode
from io import BytesIO
from pydub import AudioSegment
from pydub.silence import detect_silence
import whisper  # Open-source Whisper library

JAVASCRIPT_RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=> {
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""

# Function for recording with speech-to-text transcription
def record_and_transcribe():
    print("Start recording...")

    # Parameters for silence detection
    silence_thresh = -40  # Silence threshold in dB
    silence_duration = 1500  # Silence duration in milliseconds

    audio_data = AudioSegment.silent(duration=0)  # Start with empty audio
    transcription_file = "/content/transcription.txt"  # File to save transcription

    # Clear the transcription file at the start
    with open(transcription_file, "w") as f:
        f.write("Transcription:\n")

    # Load the Whisper model
    model = whisper.load_model("base")  # model size: tiny, base, small, medium, large

    while True:
        display(Javascript(JAVASCRIPT_RECORD))
        js_audio = output.eval_js('record(3000)')  # Record 3-second chunks
        b = b64decode(js_audio.split(',')[1])
        chunk = AudioSegment.from_file(BytesIO(b))
        audio_data += chunk  # Append the new chunk

        # Save the chunk to a temporary file for transcription
        chunk.export("/content/temp_chunk.wav", format="wav")

        # Transcribe the chunk
        transcription = transcribe_audio("/content/temp_chunk.wav", model)
        print(f"Chunk Transcription: {transcription}")

        # Save the transcription to the file
        with open(transcription_file, "a") as f:
            f.write(transcription + "\n")

        # Detect silence in the audio
        silence_ranges = detect_silence(audio_data, min_silence_len=silence_duration, silence_thresh=silence_thresh)

        # Stop recording if the last chunk is silent
        if silence_ranges and silence_ranges[-1][1] >= len(audio_data):
            print("Silence detected. Stopping recording.")
            break

    # Save the entire recorded audio
    audio_data.export("/content/continuous_audio.wav", format="wav")
    print("Recording saved as 'continuous_audio.wav'.")
    print(f"Transcription saved as '{transcription_file}'.")

# Function for transcribing audio using local Whisper
def transcribe_audio(file_path, model):
    try:
        result = model.transcribe(file_path)
        return result["text"]
    except Exception as e:
        return f"Error during transcription: {str(e)}"




Start recording

In [ ]:
record_and_transcribe()

Start recording...


/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device)


<IPython.core.display.Javascript object>

Chunk Transcription:  For example, in two weeks


<IPython.core.display.Javascript object>

Chunk Transcription:  and like


<IPython.core.display.Javascript object>

Chunk Transcription:  if I can have a good grade on that please.


<IPython.core.display.Javascript object>

Chunk Transcription: 
Silence detected. Stopping recording.
Recording saved as 'continuous_audio.wav'.
Transcription saved as '/content/transcription.txt'.


Print out Transcription File

In [ ]:
# File path
transcription_file = "/content/transcription.txt"

# Read and print the contents of the file
try:
    with open(transcription_file, "r") as f:
        transcription_text = f.read()
        print("Contents of transcription file:\n")
        print(transcription_text)
except FileNotFoundError:
    print(f"The file '{transcription_file}' does not exist.")


Contents of transcription file:

Transcription:
 For example, in two weeks
 and like
 if I can have a good grade on that please.


